<a href="https://colab.research.google.com/github/AlcirCastro/Inteligencia-Artificial-2o-Trabalho-Experimento-de-Aprendizado-Nao-Supervisionado/blob/main/Trabalho2_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install minisom

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel("StartUpsESG_0602.xlsx")

In [ ]:
df.head()

,id,StartUp,Country,Year,ESG,E,S,G,Raised,distr_tk,...,FISCALFRD,FINACIFRD,FOREIGINVEST,FOREIGDP,INTERPAY,percmath,GINI,MAKTEFF,PERCTECH,DEVCOUNT
0,12316,VenusEnergy,Lithuania,2018,0.010,0.008,0.002,0.000,5986183.0,0.27,...,96.7,70,1.299842e+09,2.420713,3.476571,4.26941,35.7,4.642995,3.11263,1
1,23416,GreenEnergyCoin,Netherlands,2018,0.018,0.010,0.006,0.002,0.0,0.00,...,88.2,80,-3.614670e+11,-39.565294,2.264509,6.37706,28.1,5.408080,2.82506,1
2,1553,HydroMiner,Austria,2017,0.014,0.010,0.000,0.004,3500000.0,0.00,...,81.1,70,-2.855524e+10,-6.277635,3.644302,6.07867,30.8,4.897589,4.38647,1
3,17028,PowerCorp,"Hong Kong SAR, China",2018,0.008,0.006,0.000,0.002,0.0,0.00,...,100.0,90,9.703626e+10,26.828457,0.050000,0.00000,0.0,5.705542,0.00000,1
4,8483,Emyron Coin,United States,2018,0.017,0.000,0.011,0.006,0.0,0.28,...,54.8,80,2.614820e+11,1.268600,11.816379,7.20920,41.4,5.209139,3.78213,1


In [ ]:
df2 = df[["Raised","ESG","E","S","G","Country"]]
df2.describe()

,Raised,ESG,E,S,G
count,6.541000e+03,6541.000000,6541.000000,6541.000000,6541.000000
mean,4.091069e+06,0.000894,0.000168,0.000471,0.000310
std,2.182035e+07,0.002945,0.001605,0.001987,0.001353
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,1.116000e+06,0.000000,0.000000,0.000000,0.000000
max,1.000000e+09,0.060000,0.080000,0.060000,0.060000


In [ ]:
media_esg = df2.groupby("Country")["ESG"].mean()
for index, row in df2.iterrows():
    if row['ESG'] == 0:
        df2.at[index, 'ESG'] = media_esg[row['Country']]


media_e = df2.groupby("Country")["E"].mean()
for index1, row in df2.iterrows():
    if row['E'] == 0:
        df2.at[index1, 'E'] = media_e[row['Country']]


media_s = df2.groupby("Country")["S"].mean()
for index2, row in df2.iterrows():
    if row['S'] == 0:
        df2.at[index2, 'S'] = media_s[row['Country']]


media_g = df2.groupby("Country")["G"].mean()
for index3, row in df2.iterrows():
    if row['G'] == 0:
        df2.at[index3, 'G'] = media_g[row['Country']]

df2.describe()

,Raised,ESG,E,S,G
count,6.541000e+03,6541.000000,6541.000000,6541.000000,6541.000000
mean,4.091069e+06,0.001649,0.000329,0.000884,0.000585
std,2.182035e+07,0.002742,0.001598,0.001902,0.001301
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000802,0.000084,0.000396,0.000240
50%,0.000000e+00,0.000871,0.000148,0.000464,0.000297
75%,1.116000e+06,0.001099,0.000193,0.000508,0.000346
max,1.000000e+09,0.060000,0.080000,0.060000,0.060000
